# Trabajo práctico 1
## Alumnos

* Garibaldi Anele
* Vergara Marcos

### Enunciado con respuesta escrita

5) Modifique el servidor para que preste el servicio de **echo (vea RFC862)**.

### Enunciado con respuesta codificada

In [ ]:
#include <sys/types.h>
#include <sys/socket.h>
#include <netinet/in.h>
#include <string.h>
#include <stdlib.h>
#include <stdio.h>
#include <fcntl.h> // for open
#include <unistd.h> // for close
#include <unistd.h>
#define PORTNUMBER  12345


int siguienteCaracter(char caracter){
    /*
    Devuelve el siguiente caracter de la secuencia RFC_864
    */
    return caracter > 126? 33 : caracter + 1;
}


void RFC_864(int ns){
    /*
    Genera caracteres segun la norma RFC_864
    */
    int i;
    char caracter;
    char salto = '\n';
    int inicio_cadena = 32;
    while(1){
        inicio_cadena = siguienteCaracter(inicio_cadena);
        caracter = inicio_cadena;
        for(i = 0; i < 72; i++){
            write(ns, &caracter, 1);
            caracter=siguienteCaracter(caracter);
        };
        write(ns, &salto, 1);
    }
}


void serve_conections(int s, struct sockaddr_in direcc){
    /*
    bind(): Se encarga de conectar al socket en un puerto determinado.
    listen(): Escucha x cantidad de conecciones del socket indicado.
    accept(): Por último la función accept que nos servirá para estar a la escucha y permitir que algún cliente se conecte tras utilizar la función connect();
    recv(): Permite recibir datos en un socket por parte de un cliente.
    write(): Escribe n bytes del buffer en la salida estandar del file descriptor proporcionado. 
    close(): Cierra la coneccion del socket.
    */
    char buf[10];
    int n, ns, len;
    len = sizeof(struct sockaddr_in);
    bind(s, (struct sockaddr *) &direcc, len);
    listen(s, 5);
    while(1){
        ns = accept(s, (struct sockaddr *) &direcc, &len);
        RFC_864(ns);
        close(ns);
    }
}


int main(void){
    /*
   Socket TCP v1.0 Garibaldi Anele, Vergara Marcos.
   Notas de proyecto:

   Socket:
   - socket(&direcc, sizeof(direcc), 0): el tercer parametro es protocolo utilizado si pones 0 lo identifica automaticamente.
   - bzero(&direcc, sizeof(direcc)):setea la variable direcc a 0. al ser una Struct se utiliza esta funcion pero está DEPRECATED. Uso memset() que hace lo mismo pero lleva un 0 como tercer parametro si queres setear todos los valores de la Struct a 0.
   
   Direcc:
   - sin_family = AF_INET le dice al socket que los clientes pueden estar dentro o fuera de la misma maquina del server. La comunicacion es por red o interna, en este caso es RED. sino es AF_UNIX para socket entre procesos
   - sin_addr.s_addr = INADDR_ANY es la dirección del cliente al que queremos atender. Poniendo en ese campo INADDR_ANY, atendemos a cualquier cliente.
   - sin_port = Es el puerto en el que el socket está escuchando conecciones. En este caso el puerto es 12345
   */
   int s;
   struct sockaddr_in direcc;
   s = socket(AF_INET, SOCK_STREAM, 0);
   //bzero((char *) &direcc, sizeof(direcc));
   memset(&direcc, 0, sizeof(direcc));
   direcc.sin_family = AF_INET;
   direcc.sin_port = htons(PORTNUMBER);
   direcc.sin_addr.s_addr = htonl(INADDR_ANY);

   serve_conections(s,direcc);
   close(s);
   exit(0);
}